In [1]:
from skimage import io
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from skimage import data, draw, segmentation
from skimage.draw import rectangle
import os
import glob

## Format images
The following cells are used to format the images provided in the beginning of the project to be used by GOTURN. The model need to have access to a `data` folder in the **OTB** format (a folder for each sequence with a subfolder containing the actual images composing the video, as well as a `groundtruth_rect.txt` file containing the top left coordinate of the boudning box, as well as the width and height of the box)

To do this, we place all the 503 .bmp images and .png masks in a folder named **sequence-train**
The code will then create as many `groundtruth_rect.txt` files as the number of different sequences (`book`, `bag`, `rhino` etc)

Finnaly, we move each sequence's images into its own folder inside a subfolder `img` and with the corresponding `groundtruth_rect.txt` file

In [ ]:
def create_bounding_box(img, mask, margin, color):
    height, width   = img.shape[0], img.shape[1]
    ones            = np.where(mask==255)
    
    # Get the max coordinates of the mask
    x_coords        = np.array(ones[0])
    y_coords        = np.array(ones[1])
    max_left_idx    = np.argmin(x_coords)
    max_right_idx   = np.argmax(x_coords)
    max_up_idx      = np.argmax(y_coords)
    max_down_idx    = np.argmin(y_coords)

    # Create a mask for the bounding box
    top_left        = (max(0, x_coords[max_left_idx] - margin), min(height, y_coords[max_down_idx] - margin))
    bottom_right    = (min(width, x_coords[max_right_idx] + margin), max(0, y_coords[max_up_idx] + margin))
    
    y_min, x_min = top_left
    y_max, x_max = bottom_right
    bbox_width = x_max - x_min
    bbox_height = y_max - y_min
        
    return [x_min, y_min, bbox_width, bbox_height]

In [4]:
def get_base_name(filename):
    # Extract the base name (before the underscore) from the filename
    return filename.split('_')[0]

def get_sorted_files(img_folder):
    # Get all bmp and png files in the folder
    bmp_files = [f for f in os.listdir(img_folder) if f.endswith(".bmp")]
    png_files = [f for f in os.listdir(img_folder) if f.endswith(".png")]
    
    # Sort both lists by the base name
    bmp_files.sort(key=lambda f: get_base_name(f))
    png_files.sort(key=lambda f: get_base_name(f))
    
    return bmp_files, png_files

In [5]:
import re
from collections import defaultdict
import shutil

def extract_name(filename):
    match = re.match(r"(.*)-\d+\.bmp", filename)
    return match.group(1) if match else None

In [6]:
img_folder  = "sequences-train"
margin      = 0

img_names, mask_names = get_sorted_files(img_folder)

grouped_files = defaultdict(list)

for img in img_names:
    name = extract_name(img)
    if name:
        grouped_files[name].append(img)

# Process each group separately
for name, images in grouped_files.items():

    with open(f"{name}_groundtruth_rect.txt", 'a') as f:
        for img in images:
            mask = img.replace(".bmp", ".png")
            if mask not in mask_names:
                continue  # Skip if no matching mask

            curr_im = io.imread(os.path.join(img_folder, img))
            curr_mask = io.imread(os.path.join(img_folder, mask))
            [x_min, y_min, bbox_width, bbox_height] = create_bounding_box(curr_im, curr_mask, margin, (0, 1, 0))
            f.write(f"{x_min},{y_min},{bbox_width},{bbox_height}\n")

In [ ]:
groundtruth_files = {}
all_files = os.listdir(img_folder)

for file in all_files:
    if file.endswith("groundtruth_rect.txt"):
        name = extract_name(file)  # Try to extract `{some_name}`
        if name:
            groundtruth_files[name] = file

# Process each group
for name, images in grouped_files.items():
    # Create `{some_name}` folder and `img` subfolder
    name_folder = name
    img_subfolder = os.path.join(name_folder, "img")
    os.makedirs(img_subfolder, exist_ok=True)

    # Move images to `img` subfolder
    for img in images:
        shutil.move(os.path.join(img_folder, img), os.path.join(img_subfolder, img))

    # Move the existing groundtruth file (if it exists)
    if name in groundtruth_files:
        shutil.move(os.path.join(img_folder, groundtruth_files[name]), os.path.join(name_folder, "groundtruth.txt"))

## Process images:
The following code processes the images from a `sequences` folder (it should contain the 503 mask images provided in the beginning of the project)
It can either add horizontal and vertical black lines on the images to simulate occlusion, or randomly change the contrast/ lighting condition of the images, by setting the mode to `'lines'` or `'contrast'` respectively.

In [ ]:
import cv2
import random

def process_images(sequence_folder, mode='lines'):
    # Iterate over each subfolder in the sequence folder
    for subfolder in os.listdir(sequence_folder):
        subfolder_path = os.path.join(sequence_folder, subfolder)
        img_folder = os.path.join(subfolder_path, "img")

        if not os.path.isdir(img_folder):
            continue

        for img_file in os.listdir(img_folder):
            if img_file.endswith(".bmp"):
                img_path = os.path.join(img_folder, img_file)
                output_path = os.path.join(os.path.join(subfolder_path, "processed_img"), f"processed_{img_file}")

                # Read the image
                image = cv2.imread(img_path)
                if image is None:
                    continue

                if mode=='lines':
                    # Get image dimensions
                    h, w, _ = image.shape

                    # Draw centered horizontal and vertical black lines
                    thick = 5
                    cv2.line(image, (w // 2, 0), (w // 2, h), (0, 0, 0), thick)  # Vertical line
                    cv2.line(image, (0, h // 2), (w, h // 2), (0, 0, 0), thick)  # Horizontal line
                    
                    # Save modified image
                    cv2.imwrite(output_path, image)
                
                if mode='contrast':
                    alpha = random.uniform(0.1, 1.9)
                    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)  # Convert to LAB color space
                    l, a, b = cv2.split(lab)  # Split channels
                    # Apply contrast scaling to L-channel (lightness)
                    l = cv2.convertScaleAbs(l, alpha=alpha, beta=0)
                    # Merge back and convert to BGR
                    lab = cv2.merge((l, a, b))
                    adjusted_contrast = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
                    adjusted_contrast = np.clip(alpha * image, 0, 255).astype(np.uint8)

                    # Save modified image
                    cv2.imwrite(output_path, adjusted_contrast)

                print(f"Processed: {output_path}")

# Example usage
process_images("sequences", mode='contrast')